In [1]:
from theano import *
from lasagne.layers import EmbeddingLayer, InputLayer, get_output
import lasagne
import lasagne.layers
import theano.tensor as T
import theano
import numpy as np

Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
from wordvecs import WordVectors

wordvectors = WordVectors(fname="../GoogleNews-vectors-negative300.bin", negvectors=False)

from sentiment_sents import Sentiment

# just load the sentences from the CNN system
sentiment = Sentiment("prevwork/CNN_sentence/")

In [3]:
len(wordvectors.vectors)

3000000

In [4]:
class SentimentExp(object):
    
    def __init__(self, train_X, train_Y, wordvecs=wordvectors):
        self.train_X = train_X
        self.train_Y = train_Y
        self.wordvecs = wordvecs
        
        self.input_size = 10
        self.batch_size = 10
        
        self.learning_rate = .01
        self.momentum = .9
        
        self.train_X_rep = np.array([[self.getRep(x)] for x in self.train_X])
        
        self._setup()
        
    def getRep(self, sent):
        ret = []
        for i in xrange(self.input_size):
            if i < len(sent):
                ret.append(self.wordvecs[sent[i]])
            else:
                ret.append(np.zeros(self.wordvecs.vector_size))
        return np.matrix(ret).reshape((1, self.input_size, self.wordvecs.vector_size))

    def _setup(self):
        self.x_batch = T.tensor4('x')
        self.y_batch = T.ivector('y')
        
        self.input_l = lasagne.layers.InputLayer((self.batch_size, 1, self.input_size, self.wordvecs.vector_size))
        
        self.first_l = lasagne.layers.Conv2DLayer(
            self.input_l,
            num_filters=100,
            filter_size=(2, self.wordvecs.vector_size),
            name='conv1',
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.first_l_max = lasagne.layers.MaxPool2DLayer(
            self.first_l,
            pool_size=(1,9)
        )
        
        self.hidden1_l = lasagne.layers.DenseLayer(
            self.first_l_max,
            num_units=50,
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.hidden1_l_drop = lasagne.layers.DropoutLayer(
            self.hidden1_l,
            p=.25,
        )
        
        
        self.out_l = lasagne.layers.DenseLayer(
            self.hidden1_l_drop,
            num_units=1,
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.output = lasagne.layers.get_output(self.out_l, self.x_batch)
        
        self.loss_vec_old = (self.output.reshape((self.output.size,)) - self.y_batch) ** 2
        self.output_diff = T.neq((self.output.flatten() > .5),(self.y_batch > .5)).sum()
        self.loss_vec = lasagne.objectives.binary_crossentropy(T.clip(self.output.reshape((self.output.size,)), .01, .99), self.y_batch)
        
        self.all_params = lasagne.layers.get_all_params(self.out_l)
        
        self.updates = lasagne.updates.adagrad(self.loss_vec.mean(), self.all_params, .01)
        #self.updates = lasagne.updates.apply_momentum(self.updates_adagrad)
        
        self.train_func = theano.function(
            [self.x_batch, self.y_batch],
            [self.loss_vec.mean(), self.loss_vec],
            updates=self.updates,
        )
        
        self.loss_func = theano.function(
            [self.x_batch, self.y_batch],
            [self.loss_vec.sum(), self.loss_vec, self.output_diff],
        )
        
    def train(self):
        for s in xrange(0, len(self.train_X_rep), self.batch_size):
            X_vals = np.array(self.train_X_rep[s:(s + self.batch_size)])
            y_vals = np.array(self.train_Y[s:(s + self.batch_size)]).astype('int32')
            loss, _ = self.train_func(X_vals, y_vals)
            
    def test_loss(self, test_X, test_Y):
        test_X_rep = np.array([[self.getRep(x)] for x in test_X])
        loss_sum = 0.0
        wrong = 0.0
        for s in xrange(0, len(test_X_rep), self.batch_size):
            X_vals = np.array(self.train_X_rep[s:(s + self.batch_size)])
            y_vals = np.array(self.train_Y[s:(s + self.batch_size)]).astype('int32')
            loss, _, output_diff = self.loss_func(X_vals, y_vals)
            wrong += output_diff
            loss_sum += loss
        return loss_sum / len(test_X_rep), wrong / len(test_X_rep)
    
experiment = SentimentExp(sentiment.train_X, sentiment.train_Y)

/home/matthew/.virtualenvs/nlp-convnet/lib/python2.7/site-packages/lasagne/init.py:86: UserWarning: The uniform initializer no longer uses Glorot et al.'s approach to determine the bounds, but defaults to the range (-0.01, 0.01) instead. Please use the new GlorotUniform initializer to get the old behavior. GlorotUniform is now the default for all layers.
  warnings.warn("The uniform initializer no longer uses Glorot et al.'s "
/home/matthew/.virtualenvs/nlp-convnet/lib/python2.7/site-packages/lasagne/layers/helper.py:69: UserWarning: get_all_layers() has been changed to return layers in topological order. The former implementation is still available as get_all_layers_old(), but will be removed before the first release of Lasagne. To ignore this warning, use `warnings.filterwarnings('ignore', '.*topo.*')`.
  warnings.warn("get_all_layers() has been changed to return layers in "


In [17]:
experiment.input_l.shape

(10, 1, 10, 300)

In [5]:
experiment.test_loss(sentiment.test_X, sentiment.test_Y)

(2.1193660567112711, 0.49788980070339978)

In [12]:
for i in xrange(15):
    experiment.train()

In [13]:
experiment.test_loss(sentiment.train_X, sentiment.train_Y)

(2.2979136422452391, 0.49788980070339978)

In [14]:
experiment.test_loss(sentiment.test_X, sentiment.test_Y)

(2.2979136422452391, 0.49788980070339978)

In [9]:
test_func = theano.function(
    [experiment.x_batch, experiment.y_batch],
    [experiment.loss_vec.mean(), experiment.loss_vec, experiment.output, 
     T.grad(experiment.loss_vec.mean(), experiment.out_l.get_params()[0]),
     experiment.out_l.get_params()[0], experiment.y_batch, 
     #lasagne.layers.get_output(experiment.first_l, experiment.x_batch)
    ]
)

In [15]:
test_func(np.array(experiment.train_X_rep[0:10]),np.array(experiment.train_Y[0:10]).astype('int32'))

[array(1.848098275907337),
 array([ 0.01005034,  4.60517019,  4.60517019,  0.01005034,  0.01005034,
         4.60517019,  0.01005034,  0.01005034,  0.01005034,  4.60517019]),
 array([[-0.99997387],
        [-0.99900812],
        [-0.90591863],
        [-0.99999876],
        [-0.99996585],
        [-0.99980893],
        [-0.99999528],
        [-0.99999882],
        [-0.99995047],
        [-0.9999158 ]]),
 array([[ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        

In [26]:
gg_k = theano.shared(0.)
gg_i = T.iscalar('x')

gg_res, gg_update = theano.scan(lambda: {gg_k:(gg_k + 1)}, sequences=[range(gg_i)])

TypeError: range() integer end argument expected, got TensorVariable.

In [21]:
gg_update

OrderedUpdates([(<TensorType(float64, scalar)>, Subtensor{int64}.0)])

In [38]:
a1 = T.matrix()
f1 = theano.function([a1], [a1.shape])

In [45]:
t1 = np.array([[1,2,3,0,0],[4,5,6,7,0],[2,2,2,2,2]])

In [46]:
t1.dtype

dtype('int64')

In [39]:
f1(t1)

AttributeError: ('Bad input argument to theano function with name "<ipython-input-38-a285aa41773e>:2"  at index 0(0-based)', "'float' object has no attribute 'dtype'")